In [1]:
# to manipulate dataframes
import pandas as pd

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# for natural language processing: named entity recognition
import spacy
from collections import Counter

# for natural language processing: sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import word_tokenize, sent_tokenize

# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = ['kiva']

/Users/ednalyndedios/.pyenv/versions/3.8.2/envs/kln382/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ednalyndedios/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ednalyndedios/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# get the data; text data MUST be in the first column
df = pd.read_csv('../data/input/kiva_loans.csv')

In [3]:
df.head()

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


In [4]:
# removes null values
df = df.loc[df.use.notnull()]

In [5]:
df = df[['id', 'funded_amount', 'loan_amount', 'activity', 'sector', 'use', 'country_code', 'borrower_genders', 'date']]

In [6]:
df.shape

(666973, 9)

In [7]:
df = df.sample(5000, random_state=493).reset_index(drop=True)

In [8]:
df.shape

(5000, 9)

In [9]:
def clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english')+ ADDITIONAL_STOPWORDS
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

def get_bigrams(content):
    """
    Takes in a list of words and returns a dataframe with the top 20 bigrams.
    """
    bigrams = (pd.Series(nltk.ngrams(content, 2)).value_counts())[:10].to_frame().reset_index()
    bigrams.columns=['bigram', 'count']
    return bigrams

def get_trigrams(content):
    """
    Takes in a list of words and returns a dataframe with the top 20 trigrams.
    """
    trigrams = (pd.Series(nltk.ngrams(content, 3)).value_counts())[:10].to_frame().reset_index()
    trigrams.columns=['trigram', 'count']
    return trigrams

In [10]:
# converts to a list of clean tokens
content = clean(''.join(str(df.use.tolist())))

# N-Gram Ranking

In [11]:
# TODO: push to S3
# exports to a flat file
get_bigrams(content).to_csv('../data/output/output_data_ngram_bigrams.csv', index=False)

In [12]:
# TODO: push to S3
# exports to a flat file
get_trigrams(content).to_csv('../data/output/output_data_ngram_trigrams.csv', index=False)

# Named-Entity Recognition

In [13]:
def get_entities(entities, ent_type):
    group_list = []
    for entity in entities:
        if entity.label_ == ent_type:
            group_list.append(entity.text)
    df_entities = pd.DataFrame(Counter(group_list).most_common(20))
    df_entities.columns=['entity', 'count']
    return df_entities

In [14]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "textcat"])

In [15]:
%%timeit
df['tokens'] = [nlp(''.join(str(row))) for row in df.use]

9.36 s ± 1.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
df['entities'] = [tokens.ents for tokens in df.tokens]

In [17]:
entities = df.entities.tolist()

In [18]:
entities = [item for sublist in (df['entities'].tolist()) for item in sublist]

In [19]:
target_entities = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT']

for item in target_entities:
    file_name = '../data/output/output_data_ner_{}.csv'.format(item.lower())
    
    try:
        get_entities(entities, item).to_csv(file_name, index=False)
    except:
        pass

In [20]:
df = df.drop(columns=['tokens', 'entities'])

# Sentiment Analysis

In [21]:
nltk.download('vader_lexicon')
sentiment = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ednalyndedios/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
df['polarity_score'] = df.iloc[:,0].apply(lambda x: sentiment.polarity_scores(str(x))['compound'])
df['neutral'] = df.iloc[:,0].apply(lambda x:sentiment.polarity_scores(str(x))['neu'])
df['negative'] = df.iloc[:,0].apply(lambda x:sentiment.polarity_scores(str(x))['neg'])
df['positive'] = df.iloc[:,0].apply(lambda x:sentiment.polarity_scores(str(x))['pos'])

In [23]:
df.loc[(df['polarity_score'] > 0.78), 'sentiment'] = 'Strongly Positive'
df.loc[(df['polarity_score'] >= 0.30) & (df['polarity_score'] <= 0.78), 'sentiment'] = 'Moderate Positive'
df.loc[(df['polarity_score'] < 0.30) & (df['polarity_score'] >= -0.30), 'sentiment'] = 'Neutral'
df.loc[(df['polarity_score'] < -0.30) & (df['polarity_score'] >= -0.55), 'sentiment'] = 'Moderate negative'
df.loc[(df['polarity_score'] < -0.55), 'sentiment'] = 'Strongly Negative'

In [24]:
df.to_csv('../data/output/output_data_sentiment.csv', index=False)

In [25]:
df = df.drop(columns=['polarity_score', 'neutral', 'negative', 'positive', 'sentiment'])

# Topic Modeling

In [30]:
df = df.sample(1000, random_state=493).reset_index(drop=True)

In [31]:
#import nlp module
from pycaret.nlp import *

#intialize the setup
nlp = setup(data = df, target = 'use')

Description,Value
session_id,5493
# Documents,1000
Vocab Size,1003
Custom Stopwords,False


In [40]:
lda = create_model('lda', num_topics=32)

In [41]:
# label the data using trained model
lda_df = assign_model(lda)

In [42]:
# plot a model
plot_model(lda, plot='topic_distribution')

In [39]:
tuned_lda = tune_model(model = 'lda')

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 300 | Coherence: 0.8173
